In [48]:
import os
import numpy as np
import pandas as pd
import torch
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
from torch import nn, optim
from PIL import Image


In [49]:
import os
base_dir = os.path.expanduser("~/Downloads/aml-2024-feather-in-focus")

train_csv = os.path.join(base_dir, "train_images.csv")
test_csv = os.path.join(base_dir, "test_images_sample.csv")
train_dir = os.path.join(base_dir, "train_images/train_images")
test_dir = os.path.join(base_dir, "test_images/test_images")
class_names_file = os.path.join(base_dir, "class_names.npy")

class_names = np.load(class_names_file, allow_pickle=True).item()

train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)

In [74]:
import os
import numpy as np
import pandas as pd

# File paths
train_csv = "~/Downloads/aml-2024-feather-in-focus/train_images.csv"
test_csv = "~/Downloads/aml-2024-feather-in-focus/test_images_sample.csv"
train_dir = "~/Downloads/aml-2024-feather-in-focus/train_images/train_images"
test_dir = "~/Downloads/aml-2024-feather-in-focus/test_images/test_images"
class_names_file = "~/Downloads/aml-2024-feather-in-focus/class_names.npy"

# Expanding user (~) to full path
train_csv = os.path.expanduser(train_csv)
test_csv = os.path.expanduser(test_csv)
train_dir = os.path.expanduser(train_dir)
test_dir = os.path.expanduser(test_dir)
class_names_file = os.path.expanduser(class_names_file)

# Load class names
class_names = np.load(class_names_file, allow_pickle=True).item()

# Load CSV files
train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)


In [77]:
print(len(train_dir))
print(len(test_df))


83
4000


In [79]:
class BirdDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None, train=True):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform
        self.train = train

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if self.train:
            img_path = os.path.join(self.img_dir, str(self.dataframe.iloc[idx, 0]))  
            image = Image.open(img_path).convert("RGB")
            if self.transform:
                image = self.transform(image)
            
            label = int(self.dataframe.iloc[idx, 1]) - 1  
            return image, label
        else:
            img_id = str(self.dataframe.iloc[idx, 0])  
            img_path = os.path.join(self.img_dir, img_id + ".jpg")  
            image = Image.open(img_path).convert("RGB")
            if self.transform:
                image = self.transform(image)
            
            return image, img_id  


In [80]:
class BirdDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None, train=True):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform
        self.train = train

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if self.train:
            img_name = str(self.dataframe.iloc[idx, 0]) 
            if not img_name.endswith(".jpg"):           
                img_name += ".jpg"
            img_path = os.path.join(self.img_dir, img_name)   

            image = Image.open(img_path).convert("RGB")
            if self.transform:
                image = self.transform(image)
            
            label = int(self.dataframe.iloc[idx, 1]) - 1
            return image, label
        else:
            img_id = str(self.dataframe.iloc[idx, 0])  
            img_path = os.path.join(self.img_dir, img_id + ".jpg")  
            image = Image.open(img_path).convert("RGB")
            if self.transform:
                image = self.transform(image)
            
            return image, img_id


In [81]:
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [82]:
# Datasets
train_dataset = BirdDataset(train_df, train_dir, transform=train_transforms, train=True)
test_dataset = BirdDataset(test_df, test_dir, transform=test_transforms, train=False)

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [83]:
import torch
from torchvision.models import resnet50
import torch.nn as nn

weights_path = "/Users/nikolayfilipov/Downloads/resnet50-0676ba61.pth"

import os
if not os.path.exists(weights_path):
    raise FileNotFoundError(f"The weights file was not found at {weights_path}")

state_dict = torch.load(weights_path)

model = resnet50()
model.load_state_dict(state_dict)

num_classes = len(class_names)

model.fc = nn.Linear(model.fc.in_features, num_classes)

print("Model loaded successfully and modified for custom classes!")


/var/folders/4z/zhkczy51005d_lqgqgrsm5r00000gp/T/ipykernel_44589/3923748774.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(weights_path)


Model loaded successfully and modified for custom classes!


In [84]:
criterion = nn.CrossEntropyLoss()  # Cross-entropy for classification
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [85]:
import matplotlib as plt
def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader):.4f}")



In [86]:
import matplotlib.pyplot as plt

def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    epoch_losses = []  

    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        epoch_losses.append(epoch_loss)  
        print(f"Epoch {epoch + 1}, Loss: {epoch_loss:.4f}")
    
    plt.figure(figsize=(8, 6))
    plt.plot(range(1, epochs + 1), epoch_losses, marker='o', label='Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss Over Epochs')
    plt.legend()
    plt.grid(True)
    plt.show()

In [87]:
train_model(model, train_loader, criterion, optimizer, epochs=10)

FileNotFoundError: [Errno 2] No such file or directory: '/train_images/1460.jpg'

In [39]:
def evaluate_model(model, test_loader):
    model.eval()
    predictions = []
    image_names = []
    
    with torch.no_grad():
        for images, img_paths in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            
            predictions.extend(preds.cpu().numpy())
            image_names.extend(img_paths)

    return predictions, image_names


In [40]:
predictions, image_names = evaluate_model(model, test_loader)

submission_df = pd.DataFrame({"id": image_names, "label": [p + 1 for p in predictions]})  
submission_df.to_csv("submission.csv", index=False)


In [42]:
submission_df.to_csv(os.path.join(base_dir, "submission.csv"), index=False)
